In [1]:
import pickle
import tensorflow as tf
import numpy as np
import tf_util
import gym
import load_policy
import tempfile
import time
%autosave 120

Autosaving every 120 seconds


# Data Generation

In [2]:
def expert(expert_policy_file = "experts/Humanoid-v1.pkl", 
           envname = "Humanoid-v1", render = True, max_timesteps = None, num_rollouts = 20, verbose = False):
    if verbose:
        print('loading and building expert policy')
    policy_fn = load_policy.load_policy( expert_policy_file)
    if verbose:
        print('loaded and built')

    with tf.Session():
        tf_util.initialize()

        import gym
        env = gym.make( envname)
        max_steps =  max_timesteps or env.spec.timestep_limit

        returns = []
        observations = []
        actions = []
        for i in range(num_rollouts):
            if verbose:
                print('-----> iter', i)
            obs = env.reset()
            done = False
            totalr = 0.
            steps = 0
            while not done:
                action = policy_fn(obs[None,:])
                observations.append(obs)
                actions.append(action)
                obs, r, done, _ = env.step(action)
#                 print("obs",np.array(obs).shape)
                totalr += r
                steps += 1
                if render:
                    env.render()
                if verbose:
                    if steps % 100 == 0: print("%i/%i"%(steps, max_steps))
                if steps >= max_steps:
                    break
            returns.append(totalr)

        if verbose:
            print('returns', returns)
            print('mean return', np.mean(returns))
            print('std of return', np.std(returns))

        expert_data = {'observations': np.array(observations),
                       'actions': np.array(actions)}
        return expert_data

## training data

In [4]:
start = time.time()
expert_data = expert(expert_policy_file = "experts/Humanoid-v1.pkl", 
           envname = "Humanoid-v1", render = False, max_timesteps = None, num_rollouts = 100, verbose = False)
end = time.time()

print("time taken = ", (end-start),"s")

obs (1, 376) (1, 376)
Instructions for updating:
Please use tf.global_variables instead.


[2017-10-23 19:32:03,921] From /media/afakharany93/Common/Online_courses/CS294-berkeleydeeprlcourse/homework/hw1/tf_util.py:91: all_variables (from tensorflow.python.ops.variables) is deprecated and will be removed after 2017-03-02.
Instructions for updating:
Please use tf.global_variables instead.


Instructions for updating:
Use `tf.variables_initializer` instead.


[2017-10-23 19:32:03,947] From /home/afakharany93/anaconda3/lib/python3.6/site-packages/tensorflow/python/util/tf_should_use.py:175: initialize_variables (from tensorflow.python.ops.variables) is deprecated and will be removed after 2017-03-02.
Instructions for updating:
Use `tf.variables_initializer` instead.
[2017-10-23 19:32:03,952] Making new env: Humanoid-v1


time taken =  149.0485339164734 s


In [5]:
obs_shape = expert_data["observations"].shape
action_shape = expert_data["actions"].shape
print(obs_shape)
print(action_shape)

(100000, 376)
(100000, 1, 17)


## validation data

In [6]:
start = time.time()
val_data = expert(expert_policy_file = "experts/Humanoid-v1.pkl", 
           envname = "Humanoid-v1", render = False, max_timesteps = None, num_rollouts = 15, verbose = False)
end = time.time()

print("time taken = ", (end-start),"s")
print(val_data["observations"].shape)
print(val_data["actions"].shape)

obs (1, 376) (1, 376)
Instructions for updating:
Please use tf.global_variables instead.


[2017-10-23 19:34:41,458] From /media/afakharany93/Common/Online_courses/CS294-berkeleydeeprlcourse/homework/hw1/tf_util.py:91: all_variables (from tensorflow.python.ops.variables) is deprecated and will be removed after 2017-03-02.
Instructions for updating:
Please use tf.global_variables instead.


Instructions for updating:
Use `tf.variables_initializer` instead.


[2017-10-23 19:34:41,483] From /home/afakharany93/anaconda3/lib/python3.6/site-packages/tensorflow/python/util/tf_should_use.py:175: initialize_variables (from tensorflow.python.ops.variables) is deprecated and will be removed after 2017-03-02.
Instructions for updating:
Use `tf.variables_initializer` instead.
[2017-10-23 19:34:41,488] Making new env: Humanoid-v1


time taken =  23.435084581375122 s
(15000, 376)
(15000, 1, 17)


# Immitation learning Model

## Model definition

In [7]:
def deepnn(x):
    hidden1_units = 100
    hidden2_units = 50
    # Hidden 1
    with tf.name_scope('hidden1'):
        weights = tf.Variable(
            tf.truncated_normal([obs_shape[1], hidden1_units],
                                stddev=1.0 / np.sqrt(float(obs_shape[1]))),
            name='weights')
        biases = tf.Variable(tf.zeros([hidden1_units]),
                             name='biases')
        hidden1 = tf.nn.relu(tf.matmul(x, weights) + biases)
    # Hidden 2
    with tf.name_scope('hidden2'):
        weights = tf.Variable(
            tf.truncated_normal([hidden1_units, hidden2_units],
                                stddev=1.0 / np.sqrt(float(hidden1_units))),
            name='weights')
        biases = tf.Variable(tf.zeros([hidden2_units]),
                             name='biases')
        hidden2 = tf.nn.relu(tf.matmul(hidden1, weights) + biases)
    # Linear
    with tf.name_scope('linear'):
        weights = tf.Variable(
            tf.truncated_normal([hidden2_units, action_shape[2]],
                                stddev=1.0 / np.sqrt(float(hidden2_units))),
            name='weights')
        biases = tf.Variable(tf.zeros([action_shape[2]]),
                             name='biases')
        output = tf.matmul(hidden2, weights) + biases
#         print(output.get_shape)

    return tf.expand_dims(output, 1)

# def loss(pred, labels):
#   labels = tf.to_int64(labels)
#   loss = tf.square(tf.subtract(labels, pred), name='square_error')
#   return tf.reduce_mean(loss, name='mean_square_error')

# def training(loss, learning_rate):
#   # Add a scalar summary for the snapshot loss.
#   tf.summary.scalar('loss', loss)
#   # Create the gradient descent optimizer with the given learning rate.
#   optimizer = tf.train.AdamOptimizer(learning_rate)
#   # Create a variable to track the global step.
#   global_step = tf.Variable(0, name='global_step', trainable=False)
#   # Use the optimizer to apply the gradients that minimize the loss
#   # (and also increment the global step counter) as a single training step.
#   train_op = optimizer.minimize(loss, global_step=global_step)
#   return train_op


In [8]:
# Create the model
x = tf.placeholder(tf.float32, [None, 376], name="input")

# Define loss and optimizer
y_ = tf.placeholder(tf.float32, [None, 1,17], name="prediction")

pred = deepnn(x)

with tf.name_scope('loss'):
    loss = tf.square(tf.subtract(y_, pred, name='square_error'))
loss = tf.reduce_mean(loss, name='mean_square_error')
tf.summary.scalar('loss', loss)

with tf.name_scope('adam_optimizer'):
    global_step = tf.Variable(0, trainable=False)
    starter_learning_rate = 1e-3
    learning_rate = tf.train.exponential_decay(starter_learning_rate, global_step,
                                               100000, 0.96, staircase=True)
    train_step = tf.train.AdamOptimizer(learning_rate).minimize(loss, global_step=global_step)

# graph_location = tempfile.mkdtemp()
# print('Saving graph to: %s' % graph_location)
# train_writer = tf.summary.FileWriter(graph_location)
# train_writer.add_graph(tf.get_default_graph())
summary_op = tf.summary.merge_all()
saver = tf.train.Saver()

## training

In [9]:
def train(expert_data, batch_size = 100, epochs = 50, train_from_start = False, verbose = True):
    with tf.Session() as sess:
        if train_from_start:
            sess.run(tf.global_variables_initializer())
        else :
            saver.restore(sess, "./tmp/model.ckpt")
        train_writer = tf.summary.FileWriter('tensorboard/train', sess.graph)
        test_writer = tf.summary.FileWriter('tensorboard/test')
        for epoch in range(epochs):
            for batch_indx in range(batch_size,obs_shape[0], batch_size):
                batch_x = expert_data["observations"][batch_indx-batch_size:batch_indx,:]
                batch_y = expert_data["actions"][batch_indx-batch_size:batch_indx,:,:]
    #             print(batch_x.shape)

                i = (epoch+1)*batch_indx
                if i % 100 == 0:
                    training_loss= loss.eval(feed_dict={x: batch_x, y_: batch_y})
                    summary = summary_op.eval(feed_dict={x: batch_x, y_: batch_y})
                    train_writer.add_summary(summary, i)
                if verbose and (i %500 == 0):
                    print('epoch %d,step %d, training loss %g' % (epoch,i, training_loss))
                train_step.run(feed_dict={x: batch_x, y_: batch_y})

            valid_loss = loss.eval(feed_dict={x:val_data["observations"], y_:val_data["actions"] })
            summary = summary_op.eval(feed_dict={x:val_data["observations"], y_:val_data["actions"] })
            test_writer.add_summary(summary, i)
            print('epoch %d,training loss %g ,test loss %g' % (epoch,training_loss,valid_loss))
            save_path = saver.save(sess, "./tmp/model.ckpt")

In [10]:
train(expert_data, batch_size = 100, epochs = 50, train_from_start = True)

epoch 0,step 500, training loss 50.7585
epoch 0,step 1000, training loss 19.1259
epoch 0,step 1500, training loss 14.6506
epoch 0,step 2000, training loss 6.25772
epoch 0,step 2500, training loss 4.09501
epoch 0,step 3000, training loss 3.55589
epoch 0,step 3500, training loss 2.84298
epoch 0,step 4000, training loss 1.69518
epoch 0,step 4500, training loss 1.46496
epoch 0,step 5000, training loss 1.80301
epoch 0,step 5500, training loss 1.35236
epoch 0,step 6000, training loss 1.45481
epoch 0,step 6500, training loss 1.67892
epoch 0,step 7000, training loss 1.2391
epoch 0,step 7500, training loss 1.10266
epoch 0,step 8000, training loss 1.12694
epoch 0,step 8500, training loss 0.968804
epoch 0,step 9000, training loss 0.788289
epoch 0,step 9500, training loss 0.751538
epoch 0,step 10000, training loss 0.76197
epoch 0,step 10500, training loss 0.832579
epoch 0,step 11000, training loss 0.755798
epoch 0,step 11500, training loss 0.799145
epoch 0,step 12000, training loss 0.682368
epoch 

epoch 0,training loss 0.253964 ,test loss 0.229334
epoch 1,step 1000, training loss 0.214312
epoch 1,step 2000, training loss 0.257825
epoch 1,step 3000, training loss 0.215212
epoch 1,step 4000, training loss 0.14842
epoch 1,step 5000, training loss 0.137672
epoch 1,step 6000, training loss 0.177503
epoch 1,step 7000, training loss 0.156906
epoch 1,step 8000, training loss 0.135412
epoch 1,step 9000, training loss 0.146704
epoch 1,step 10000, training loss 0.152
epoch 1,step 11000, training loss 0.14181
epoch 1,step 12000, training loss 0.208634
epoch 1,step 13000, training loss 0.269657
epoch 1,step 14000, training loss 0.187545
epoch 1,step 15000, training loss 0.203567
epoch 1,step 16000, training loss 0.17717
epoch 1,step 17000, training loss 0.16808
epoch 1,step 18000, training loss 0.134658
epoch 1,step 19000, training loss 0.118711
epoch 1,step 20000, training loss 0.176448
epoch 1,step 21000, training loss 0.150967
epoch 1,step 22000, training loss 0.160976
epoch 1,step 23000,

epoch 1,step 190000, training loss 0.0948727
epoch 1,step 191000, training loss 0.105929
epoch 1,step 192000, training loss 0.12464
epoch 1,step 193000, training loss 0.11024
epoch 1,step 194000, training loss 0.126548
epoch 1,step 195000, training loss 0.144585
epoch 1,step 196000, training loss 0.0994896
epoch 1,step 197000, training loss 0.0985241
epoch 1,step 198000, training loss 0.0995925
epoch 1,step 199000, training loss 0.0861296
epoch 1,training loss 0.163615 ,test loss 0.156869
epoch 2,step 1500, training loss 0.142338
epoch 2,step 3000, training loss 0.188631
epoch 2,step 4500, training loss 0.151595
epoch 2,step 6000, training loss 0.0998412
epoch 2,step 7500, training loss 0.0889694
epoch 2,step 9000, training loss 0.12076
epoch 2,step 10500, training loss 0.107072
epoch 2,step 12000, training loss 0.0919598
epoch 2,step 13500, training loss 0.102995
epoch 2,step 15000, training loss 0.115271
epoch 2,step 16500, training loss 0.0983394
epoch 2,step 18000, training loss 0.

epoch 2,step 268500, training loss 0.112287
epoch 2,step 270000, training loss 0.160998
epoch 2,step 271500, training loss 0.0901105
epoch 2,step 273000, training loss 0.0946188
epoch 2,step 274500, training loss 0.122079
epoch 2,step 276000, training loss 0.104271
epoch 2,step 277500, training loss 0.0798851
epoch 2,step 279000, training loss 0.0835427
epoch 2,step 280500, training loss 0.121609
epoch 2,step 282000, training loss 0.0881794
epoch 2,step 283500, training loss 0.0885611
epoch 2,step 285000, training loss 0.0759322
epoch 2,step 286500, training loss 0.086363
epoch 2,step 288000, training loss 0.104937
epoch 2,step 289500, training loss 0.0842712
epoch 2,step 291000, training loss 0.0949635
epoch 2,step 292500, training loss 0.115161
epoch 2,step 294000, training loss 0.0817706
epoch 2,step 295500, training loss 0.0769295
epoch 2,step 297000, training loss 0.0686577
epoch 2,step 298500, training loss 0.0746165
epoch 2,training loss 0.138145 ,test loss 0.128189
epoch 3,step

epoch 3,step 344000, training loss 0.101429
epoch 3,step 346000, training loss 0.0693769
epoch 3,step 348000, training loss 0.0969492
epoch 3,step 350000, training loss 0.0783507
epoch 3,step 352000, training loss 0.0824322
epoch 3,step 354000, training loss 0.0742423
epoch 3,step 356000, training loss 0.0659423
epoch 3,step 358000, training loss 0.0950772
epoch 3,step 360000, training loss 0.142044
epoch 3,step 362000, training loss 0.0781179
epoch 3,step 364000, training loss 0.0807323
epoch 3,step 366000, training loss 0.0984496
epoch 3,step 368000, training loss 0.0777527
epoch 3,step 370000, training loss 0.0613568
epoch 3,step 372000, training loss 0.0701322
epoch 3,step 374000, training loss 0.102796
epoch 3,step 376000, training loss 0.0731332
epoch 3,step 378000, training loss 0.077013
epoch 3,step 380000, training loss 0.0630085
epoch 3,step 382000, training loss 0.068991
epoch 3,step 384000, training loss 0.0915554
epoch 3,step 386000, training loss 0.0719749
epoch 3,step 38

...

epoch 49,step 510000, training loss 0.0356321
epoch 49,step 515000, training loss 0.0253428
epoch 49,step 520000, training loss 0.0200976
epoch 49,step 525000, training loss 0.024123
epoch 49,step 530000, training loss 0.0225462
epoch 49,step 535000, training loss 0.0156034
epoch 49,step 540000, training loss 0.0328439
epoch 49,step 545000, training loss 0.0307006
epoch 49,step 550000, training loss 0.0285423
epoch 49,step 555000, training loss 0.0734593
epoch 49,step 560000, training loss 0.0504532
epoch 49,step 565000, training loss 0.0274979
epoch 49,step 570000, training loss 0.0407047
epoch 49,step 575000, training loss 0.0366084
epoch 49,step 580000, training loss 0.0482162
epoch 49,step 585000, training loss 0.0284693
epoch 49,step 590000, training loss 0.0453145
epoch 49,step 595000, training loss 0.0475928
epoch 49,step 600000, training loss 0.0306289
epoch 49,step 605000, training loss 0.0661533
epoch 49,step 610000, training loss 0.0426394
epoch 49,step 615000, training loss

epoch 49,step 1535000, training loss 0.0247428
epoch 49,step 1540000, training loss 0.0353681
epoch 49,step 1545000, training loss 0.0212153
epoch 49,step 1550000, training loss 0.0266726
epoch 49,step 1555000, training loss 0.0779364
epoch 49,step 1560000, training loss 0.0589258
epoch 49,step 1565000, training loss 0.0198635
epoch 49,step 1570000, training loss 0.0302378
epoch 49,step 1575000, training loss 0.0251086
epoch 49,step 1580000, training loss 0.0350064
epoch 49,step 1585000, training loss 0.0439807
epoch 49,step 1590000, training loss 0.0231892
epoch 49,step 1595000, training loss 0.0399601
epoch 49,step 1600000, training loss 0.0344164
epoch 49,step 1605000, training loss 0.0773826
epoch 49,step 1610000, training loss 0.0540992
epoch 49,step 1615000, training loss 0.0319747
epoch 49,step 1620000, training loss 0.0303878
epoch 49,step 1625000, training loss 0.0348856
epoch 49,step 1630000, training loss 0.0223345
epoch 49,step 1635000, training loss 0.0260692
epoch 49,step

epoch 49,step 2550000, training loss 0.0206567
epoch 49,step 2555000, training loss 0.0907979
epoch 49,step 2560000, training loss 0.0866535
epoch 49,step 2565000, training loss 0.0237286
epoch 49,step 2570000, training loss 0.0262616
epoch 49,step 2575000, training loss 0.0306915
epoch 49,step 2580000, training loss 0.0300195
epoch 49,step 2585000, training loss 0.0285352
epoch 49,step 2590000, training loss 0.023252
epoch 49,step 2595000, training loss 0.027367
epoch 49,step 2600000, training loss 0.0201999
epoch 49,step 2605000, training loss 0.0781298
epoch 49,step 2610000, training loss 0.0525947
epoch 49,step 2615000, training loss 0.0251466
epoch 49,step 2620000, training loss 0.0248326
epoch 49,step 2625000, training loss 0.028571
epoch 49,step 2630000, training loss 0.0220661
epoch 49,step 2635000, training loss 0.0278226
epoch 49,step 2640000, training loss 0.0271924
epoch 49,step 2645000, training loss 0.0427064
epoch 49,step 2650000, training loss 0.0331474
epoch 49,step 26

epoch 49,step 3555000, training loss 0.0756053
epoch 49,step 3560000, training loss 0.0496447
epoch 49,step 3565000, training loss 0.0255611
epoch 49,step 3570000, training loss 0.0190525
epoch 49,step 3575000, training loss 0.0270792
epoch 49,step 3580000, training loss 0.0243318
epoch 49,step 3585000, training loss 0.0306171
epoch 49,step 3590000, training loss 0.029375
epoch 49,step 3595000, training loss 0.02075
epoch 49,step 3600000, training loss 0.0214319
epoch 49,step 3605000, training loss 0.0543007
epoch 49,step 3610000, training loss 0.0382992
epoch 49,step 3615000, training loss 0.034306
epoch 49,step 3620000, training loss 0.0276144
epoch 49,step 3625000, training loss 0.0322181
epoch 49,step 3630000, training loss 0.0216196
epoch 49,step 3635000, training loss 0.0334305
epoch 49,step 3640000, training loss 0.0236171
epoch 49,step 3645000, training loss 0.0232184
epoch 49,step 3650000, training loss 0.0206343
epoch 49,step 3655000, training loss 0.0782428
epoch 49,step 366

epoch 49,step 4585000, training loss 0.0232922
epoch 49,step 4590000, training loss 0.0274827
epoch 49,step 4595000, training loss 0.0283634
epoch 49,step 4600000, training loss 0.0231025
epoch 49,step 4605000, training loss 0.0605955
epoch 49,step 4610000, training loss 0.0556325
epoch 49,step 4615000, training loss 0.0251239
epoch 49,step 4620000, training loss 0.0210293
epoch 49,step 4625000, training loss 0.0181829
epoch 49,step 4630000, training loss 0.0266065
epoch 49,step 4635000, training loss 0.0214839
epoch 49,step 4640000, training loss 0.0174116
epoch 49,step 4645000, training loss 0.0214115
epoch 49,step 4650000, training loss 0.0190269
epoch 49,step 4655000, training loss 0.0747574
epoch 49,step 4660000, training loss 0.0377651
epoch 49,step 4665000, training loss 0.0209232
epoch 49,step 4670000, training loss 0.025658
epoch 49,step 4675000, training loss 0.031885
epoch 49,step 4680000, training loss 0.0232258
epoch 49,step 4685000, training loss 0.0179148
epoch 49,step 4

In [ ]:
with tf.Session() as sess:
    saver.restore(sess, "./tmp/model.ckpt")
    prediction = pred.eval(feed_dict={x: batch_x})
    print(prediction.shape)

## Dagger

## Trying the policy

In [11]:
def try_policy(envname = "Humanoid-v1", render = True, max_timesteps = None, num_rollouts = 20, verbose = False):

    with tf.Session() as sess:
        saver.restore(sess, "./tmp/model.ckpt")

        import gym
        env = gym.make( envname)
        max_steps =  max_timesteps or env.spec.timestep_limit

        returns = []
        observations = []
        actions = []
        for i in range(num_rollouts):
            if verbose:
                print('-----> iter', i)
            obs = env.reset()
            done = False
            totalr = 0.
            steps = 0
            while not done:
#                 action = policy_fn(obs[None,:])
                len_obs = len(obs)
                batch_x = np.array(obs).reshape(1,len_obs)
                action = pred.eval(feed_dict={x: batch_x})
                observations.append(obs)
                actions.append(action)
                obs, r, done, _ = env.step(action)
                totalr += r
                steps += 1
                if render:
                    env.render()
                if verbose:
                    if steps % 100 == 0: print("%i/%i"%(steps, max_steps))
                if steps >= max_steps:
                    break
            returns.append(totalr)

        if verbose:
            print('returns', returns)
            print('mean return', np.mean(returns))
            print('std of return', np.std(returns))

        immit_data = {'observations': np.array(observations),
                       'actions': np.array(actions)}
        return immit_data

In [25]:
immit_data = try_policy(num_rollouts = 20)

INFO:tensorflow:Restoring parameters from ./tmp/model.ckpt


[2017-10-23 15:24:03,087] Restoring parameters from ./tmp/model.ckpt
[2017-10-23 15:24:03,123] Making new env: Humanoid-v1


In [27]:
print(immit_data["observations"].shape)
print(immit_data["actions"].shape)

(14397, 376)
(14397, 1, 1, 17)


## using the expert to evalute observations

In [12]:
def expert_with_obs(observation_ip,expert_policy_file = "experts/Humanoid-v1.pkl", verbose = False):
    if verbose:
        print('loading and building expert policy')
    policy_fn = load_policy.load_policy( expert_policy_file)
    if verbose:
        print('loaded and built')

    with tf.Session():
        tf_util.initialize()
        actions = []
        for i in range(observation_ip.shape[0]):
            if verbose:
                print('-----> iter', i)
            action = policy_fn(observation_ip[i:i+1,:])
            actions.append(action)

        expert_data = {'observations': observation_ip,
                       'actions': np.array(actions)}
        return expert_data

In [36]:
new_expert_data = expert_with_obs(immit_data["observations"],expert_policy_file = "experts/Humanoid-v1.pkl", verbose = True)

loading and building expert policy
obs (1, 376) (1, 376)
loaded and built
Instructions for updating:
Please use tf.global_variables instead.


[2017-10-23 19:04:40,480] From /media/afakharany93/Common/Online_courses/CS294-berkeleydeeprlcourse/homework/hw1/tf_util.py:91: all_variables (from tensorflow.python.ops.variables) is deprecated and will be removed after 2017-03-02.
Instructions for updating:
Please use tf.global_variables instead.


Instructions for updating:
Use `tf.variables_initializer` instead.


[2017-10-23 19:04:40,509] From /home/afakharany93/anaconda3/lib/python3.6/site-packages/tensorflow/python/util/tf_should_use.py:175: initialize_variables (from tensorflow.python.ops.variables) is deprecated and will be removed after 2017-03-02.
Instructions for updating:
Use `tf.variables_initializer` instead.


-----> iter 0
-----> iter 1
-----> iter 2
-----> iter 3
-----> iter 4
-----> iter 5
-----> iter 6
-----> iter 7
-----> iter 8
-----> iter 9
-----> iter 10
-----> iter 11
-----> iter 12
-----> iter 13
-----> iter 14
-----> iter 15
-----> iter 16
-----> iter 17
-----> iter 18
-----> iter 19
-----> iter 20
-----> iter 21
-----> iter 22
-----> iter 23
-----> iter 24
-----> iter 25
-----> iter 26
-----> iter 27
-----> iter 28
-----> iter 29
-----> iter 30
-----> iter 31
-----> iter 32
-----> iter 33
-----> iter 34
-----> iter 35
-----> iter 36
-----> iter 37
-----> iter 38
-----> iter 39
-----> iter 40
-----> iter 41
-----> iter 42
-----> iter 43
-----> iter 44
-----> iter 45
-----> iter 46
-----> iter 47
-----> iter 48
-----> iter 49
-----> iter 50
-----> iter 51
-----> iter 52
-----> iter 53
-----> iter 54
-----> iter 55
-----> iter 56
-----> iter 57
-----> iter 58
-----> iter 59
-----> iter 60
-----> iter 61
-----> iter 62
-----> iter 63
-----> iter 64
-----> iter 65
-----> iter 66
-----

...

-----> iter 12179
-----> iter 12180
-----> iter 12181
-----> iter 12182
-----> iter 12183
-----> iter 12184
-----> iter 12185
-----> iter 12186
-----> iter 12187
-----> iter 12188
-----> iter 12189
-----> iter 12190
-----> iter 12191
-----> iter 12192
-----> iter 12193
-----> iter 12194
-----> iter 12195
-----> iter 12196
-----> iter 12197
-----> iter 12198
-----> iter 12199
-----> iter 12200
-----> iter 12201
-----> iter 12202
-----> iter 12203
-----> iter 12204
-----> iter 12205
-----> iter 12206
-----> iter 12207
-----> iter 12208
-----> iter 12209
-----> iter 12210
-----> iter 12211
-----> iter 12212
-----> iter 12213
-----> iter 12214
-----> iter 12215
-----> iter 12216
-----> iter 12217
-----> iter 12218
-----> iter 12219
-----> iter 12220
-----> iter 12221
-----> iter 12222
-----> iter 12223
-----> iter 12224
-----> iter 12225
-----> iter 12226
-----> iter 12227
-----> iter 12228
-----> iter 12229
-----> iter 12230
-----> iter 12231
-----> iter 12232
-----> iter 12233
-----> ite

-----> iter 12714
-----> iter 12715
-----> iter 12716
-----> iter 12717
-----> iter 12718
-----> iter 12719
-----> iter 12720
-----> iter 12721
-----> iter 12722
-----> iter 12723
-----> iter 12724
-----> iter 12725
-----> iter 12726
-----> iter 12727
-----> iter 12728
-----> iter 12729
-----> iter 12730
-----> iter 12731
-----> iter 12732
-----> iter 12733
-----> iter 12734
-----> iter 12735
-----> iter 12736
-----> iter 12737
-----> iter 12738
-----> iter 12739
-----> iter 12740
-----> iter 12741
-----> iter 12742
-----> iter 12743
-----> iter 12744
-----> iter 12745
-----> iter 12746
-----> iter 12747
-----> iter 12748
-----> iter 12749
-----> iter 12750
-----> iter 12751
-----> iter 12752
-----> iter 12753
-----> iter 12754
-----> iter 12755
-----> iter 12756
-----> iter 12757
-----> iter 12758
-----> iter 12759
-----> iter 12760
-----> iter 12761
-----> iter 12762
-----> iter 12763
-----> iter 12764
-----> iter 12765
-----> iter 12766
-----> iter 12767
-----> iter 12768
-----> ite

-----> iter 13207
-----> iter 13208
-----> iter 13209
-----> iter 13210
-----> iter 13211
-----> iter 13212
-----> iter 13213
-----> iter 13214
-----> iter 13215
-----> iter 13216
-----> iter 13217
-----> iter 13218
-----> iter 13219
-----> iter 13220
-----> iter 13221
-----> iter 13222
-----> iter 13223
-----> iter 13224
-----> iter 13225
-----> iter 13226
-----> iter 13227
-----> iter 13228
-----> iter 13229
-----> iter 13230
-----> iter 13231
-----> iter 13232
-----> iter 13233
-----> iter 13234
-----> iter 13235
-----> iter 13236
-----> iter 13237
-----> iter 13238
-----> iter 13239
-----> iter 13240
-----> iter 13241
-----> iter 13242
-----> iter 13243
-----> iter 13244
-----> iter 13245
-----> iter 13246
-----> iter 13247
-----> iter 13248
-----> iter 13249
-----> iter 13250
-----> iter 13251
-----> iter 13252
-----> iter 13253
-----> iter 13254
-----> iter 13255
-----> iter 13256
-----> iter 13257
-----> iter 13258
-----> iter 13259
-----> iter 13260
-----> iter 13261
-----> ite

-----> iter 13664
-----> iter 13665
-----> iter 13666
-----> iter 13667
-----> iter 13668
-----> iter 13669
-----> iter 13670
-----> iter 13671
-----> iter 13672
-----> iter 13673
-----> iter 13674
-----> iter 13675
-----> iter 13676
-----> iter 13677
-----> iter 13678
-----> iter 13679
-----> iter 13680
-----> iter 13681
-----> iter 13682
-----> iter 13683
-----> iter 13684
-----> iter 13685
-----> iter 13686
-----> iter 13687
-----> iter 13688
-----> iter 13689
-----> iter 13690
-----> iter 13691
-----> iter 13692
-----> iter 13693
-----> iter 13694
-----> iter 13695
-----> iter 13696
-----> iter 13697
-----> iter 13698
-----> iter 13699
-----> iter 13700
-----> iter 13701
-----> iter 13702
-----> iter 13703
-----> iter 13704
-----> iter 13705
-----> iter 13706
-----> iter 13707
-----> iter 13708
-----> iter 13709
-----> iter 13710
-----> iter 13711
-----> iter 13712
-----> iter 13713
-----> iter 13714
-----> iter 13715
-----> iter 13716
-----> iter 13717
-----> iter 13718
-----> ite

-----> iter 14133
-----> iter 14134
-----> iter 14135
-----> iter 14136
-----> iter 14137
-----> iter 14138
-----> iter 14139
-----> iter 14140
-----> iter 14141
-----> iter 14142
-----> iter 14143
-----> iter 14144
-----> iter 14145
-----> iter 14146
-----> iter 14147
-----> iter 14148
-----> iter 14149
-----> iter 14150
-----> iter 14151
-----> iter 14152
-----> iter 14153
-----> iter 14154
-----> iter 14155
-----> iter 14156
-----> iter 14157
-----> iter 14158
-----> iter 14159
-----> iter 14160
-----> iter 14161
-----> iter 14162
-----> iter 14163
-----> iter 14164
-----> iter 14165
-----> iter 14166
-----> iter 14167
-----> iter 14168
-----> iter 14169
-----> iter 14170
-----> iter 14171
-----> iter 14172
-----> iter 14173
-----> iter 14174
-----> iter 14175
-----> iter 14176
-----> iter 14177
-----> iter 14178
-----> iter 14179
-----> iter 14180
-----> iter 14181
-----> iter 14182
-----> iter 14183
-----> iter 14184
-----> iter 14185
-----> iter 14186
-----> iter 14187
-----> ite

In [38]:
print(new_expert_data["observations"].shape)
print(new_expert_data["actions"].shape)

(14397, 376)
(14397, 1, 17)


## putting all together

In [13]:
num_of_inter = 10
for i in range(num_of_inter):
    start = time.time()
    immit_data = try_policy(num_rollouts = 10, render=True)
    new_expert_data = expert_with_obs(immit_data["observations"],expert_policy_file = "experts/Humanoid-v1.pkl", verbose = False)
    train(new_expert_data, batch_size = 100, epochs = 5, train_from_start = False, verbose = False)
    end = time.time()
    print("\n----> iteration %i, time %f sec" %(i, end-start))

INFO:tensorflow:Restoring parameters from ./tmp/model.ckpt


[2017-10-23 19:38:45,357] Restoring parameters from ./tmp/model.ckpt
[2017-10-23 19:38:45,423] Making new env: Humanoid-v1


obs (1, 376) (1, 376)
Instructions for updating:
Please use tf.global_variables instead.


[2017-10-23 19:38:53,408] From /media/afakharany93/Common/Online_courses/CS294-berkeleydeeprlcourse/homework/hw1/tf_util.py:91: all_variables (from tensorflow.python.ops.variables) is deprecated and will be removed after 2017-03-02.
Instructions for updating:
Please use tf.global_variables instead.


Instructions for updating:
Use `tf.variables_initializer` instead.


...

Instructions for updating:
Use `tf.variables_initializer` instead.


[2017-10-23 19:40:57,180] From /home/afakharany93/anaconda3/lib/python3.6/site-packages/tensorflow/python/util/tf_should_use.py:175: initialize_variables (from tensorflow.python.ops.variables) is deprecated and will be removed after 2017-03-02.
Instructions for updating:
Use `tf.variables_initializer` instead.


INFO:tensorflow:Restoring parameters from ./tmp/model.ckpt


[2017-10-23 19:40:57,444] Restoring parameters from ./tmp/model.ckpt


epoch 0,training loss nan ,test loss 1.72604
epoch 1,training loss nan ,test loss 1.51866
epoch 2,training loss nan ,test loss 1.558
epoch 3,training loss nan ,test loss 1.62117
epoch 4,training loss nan ,test loss 1.61917

----> iteration 9, time 14.381261 sec
